## 난임 환자 대상 임신 성공 여부 예측

### LGAimers 6th 온라인 해커톤

Import

In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

### Data Load

In [2]:
# 데이터 로드
IVF_train = pd.read_csv('../data/IVF_train_dataset_50.csv')
IVF_test = pd.read_csv('../data/IVF_test_dataset_50.csv')

In [3]:
# ID 열을 제외한 특성과 타겟 변수 분리
IVF_X = IVF_train.drop(['임신_성공_여부', 'ID'], axis=1)
IVF_y = IVF_train['임신_성공_여부']

In [4]:
print(f"IVF_X shape: {IVF_X.shape}")
print(f"IVF_test shape: {IVF_test.drop('ID', axis=1).shape}")

IVF_X shape: (202130, 61)
IVF_test shape: (87891, 61)


### 인코딩 

In [5]:
IVF_categorical_columns = [
    "난자_출처",
    "정자_출처",
    "난자_기증자_나이",
    "정자_기증자_나이"
]

In [6]:
# 모든 범주형 변수를 문자열로 변환
IVF_X[IVF_categorical_columns] = IVF_X[IVF_categorical_columns].astype(str)
IVF_test[IVF_categorical_columns] = IVF_test[IVF_categorical_columns].astype(str)

# OrdinalEncoder를 사용하여 범주형 변수 인코딩
IVF_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

IVF_X[IVF_categorical_columns] = IVF_encoder.fit_transform(IVF_X[IVF_categorical_columns])
IVF_test[IVF_categorical_columns] = IVF_encoder.transform(IVF_test[IVF_categorical_columns])

## Modeling

In [7]:
# 데이터 분할
IVF_X_train, IVF_X_test, IVF_y_train, IVF_y_test = train_test_split(IVF_X, IVF_y, test_size=0.2, random_state=42)

### IVF 데이터

In [8]:
import optuna
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

# 목적 함수 정의
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 5000),
        'num_leaves': trial.suggest_int('num_leaves', 1000, 5000),
        'learning_rate': trial.suggest_float('learning_rate', 0.00005, 0.5),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 500),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 200.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 200.0),

        'random_state': 42,
        'n_jobs': -1,
        'metric': 'auc',
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'verbose': -1
    }

    model = lgb.LGBMClassifier(**param)
    model.fit(IVF_X_train, IVF_y_train)
    
    y_pred_proba = model.predict_proba(IVF_X_test)[:, 1]
    
    auc = roc_auc_score(IVF_y_test, y_pred_proba)
    return auc

# Optuna 스터디 생성 및 최적화 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=800)

# 최적의 하이퍼파라미터 출력
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2025-02-22 23:42:47,148] A new study created in memory with name: no-name-4617d734-6ddd-4af9-83bd-c1073c977b8d
[I 2025-02-22 23:42:50,488] Trial 0 finished with value: 0.733163727519188 and parameters: {'n_estimators': 3695, 'num_leaves': 3456, 'learning_rate': 0.1676832820430354, 'min_child_samples': 383, 'reg_alpha': 55.02094451698953, 'reg_lambda': 43.0684133882276}. Best is trial 0 with value: 0.733163727519188.
[I 2025-02-22 23:42:52,004] Trial 1 finished with value: 0.7318734488127792 and parameters: {'n_estimators': 900, 'num_leaves': 2751, 'learning_rate': 0.14186562356100704, 'min_child_samples': 10, 'reg_alpha': 128.48466710717406, 'reg_lambda': 143.5321857997496}. Best is trial 0 with value: 0.733163727519188.
[I 2025-02-22 23:42:53,268] Trial 2 finished with value: 0.7325226910254033 and parameters: {'n_estimators': 1213, 'num_leaves': 1203, 'learning_rate': 0.27218480041995335, 'min_child_samples': 215, 'reg_alpha': 92.27488795247467, 'reg_lambda': 110.34582999064682}. 

KeyboardInterrupt: 

In [ ]:
# import optuna
# import lightgbm as lgb
# from sklearn.metrics import roc_auc_score

# # 목적 함수 정의
# def objective(trial):
#     param = {
#         'n_estimators': trial.suggest_int('n_estimators', 1000, 8000),
#         'max_depth': trial.suggest_int('max_depth', 3, 300),
#         'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True),
#         'num_leaves': trial.suggest_int('num_leaves', 2, 2048),
#         'min_child_samples': trial.suggest_int('min_child_samples', 1, 100),
#         'subsample': trial.suggest_float('subsample', 0.6, 1.0),
#         'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0, log=True),
#         'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0, log=True),
        
#         'boosting_type' : 'gbdt',
#         'random_state': 42,
#         'objective': 'binary',
#         'metric': 'auc',
#         'verbose': -1,   
#         'njobs': -1
#     }

#     model = lgb.LGBMClassifier(**param)
#     model.fit(IVF_X_train, IVF_y_train)
    
#     y_pred_proba = model.predict_proba(IVF_X_test)[:, 1]
    
#     auc = roc_auc_score(IVF_y_test, y_pred_proba)
#     return auc

# # Optuna 스터디 생성 및 최적화 실행
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=800)

# # 최적의 하이퍼파라미터 출력
# print("Best trial:")
# trial = study.best_trial
# print(f"  Value: {trial.value}")
# print("  Params: ")
# for key, value in trial.params.items():
#     print(f"    {key}: {value}")

.